In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

2022-03-01 16:10:13.397950: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-01 16:10:13.397978: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Load data

In [2]:
cf15= pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/cf15.csv', low_memory=False, header=None)
gf25= pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/gf25.csv', low_memory=False, header=None)
gm01= pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/gm01.csv', low_memory=False, header=None)
nc46= pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/nc46.csv', low_memory=False, header=None)
nc74_jan= pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/nc74_jan.csv', low_memory=False, header=None)
nf11= pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/nf11.csv', low_memory=False, header=None)
wc16= pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/wc16.csv', low_memory=False, header=None)
wc41= pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/wc41.csv', low_memory=False, header=None)
wo36= pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/wo36.csv', low_memory=False, header=None)


In [3]:
val_1 = pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/go22.csv', low_memory=False, header=None)
val_2 = pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/kf15.csv', low_memory=False, header=None)
val_3 = pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/nf28.csv', low_memory=False, header=None)

In [4]:
frames =[cf15, nc46, nc74_jan, nf11, wc16, wc41, wo36]
frames_2=[val_1, val_2]
data = pd.concat(frames)
val = pd.concat(frames_2)

In [5]:
def snip(data):
    time= data.iloc[:,0]
    time=time.map(lambda x: str(x[6:]))
    time=time.map(lambda x: float(x))

    interface= data.iloc[:,-14]
    interface=interface.map(lambda x: str(x[15:]))
    interface=interface.map(lambda x: str(x[:-1]))

    jitter=data.iloc[:, -13]
    jitter=jitter.map(lambda x: str(x))
    jitter=jitter.map(lambda x: str(x[8:]))
    jitter=jitter.map(lambda x: str(x[:-1]))
    jitter=jitter.map(lambda x: float(x))

    latency= data.iloc[:,-12]
    latency=latency.map(lambda x: str(x))
    latency=latency.map(lambda x: str(x[9:]))
    latency=latency.map(lambda x: str(x[:-1]))
    latency=latency.map(lambda x: float(x))

    inband= data.iloc[:,-15]
    inband=inband.map(lambda x: str(x))
    inband=inband.map(lambda x: str(x[22:]))
    inband=inband.map(lambda x: str(x[:-5]))
    inband=inband.map(lambda x: float(x))

    outband= data.iloc[:, -5]
    outband=outband.map(lambda x: str(x))
    outband=outband.map(lambda x: x.split('"')[1])
    outband=outband.map(lambda x: str(x[:-4]))
    outband=outband.map(lambda x: float(x))

    packet=data.iloc[:,-4]
    packet=packet.map(lambda x: str(x))
    packet=packet.map(lambda x: str(x[12:]))
    packet=packet.map(lambda x: str(x[:-2]))
    packet=packet.map(lambda x: float(x))

    train=pd.concat([time, inband, outband, latency, packet, jitter], axis=1)

    return train

In [6]:
def labels(data):
    status=data.iloc[:,-2]
    status=status.map(lambda x: str(x))
    status=status.map(lambda x: str(x[8:]))
    status=status.map(lambda x: str(x[:-1]))

    status[:] = np.where(status=="up", 1,0)
    status=status.map(lambda x: float(x))
    status=status.rolling(20).mean()
    status=status.map(np.floor)

    return status

In [7]:
def preproccess(data):
    train=snip(data)
    train=train.iloc[:-700, :]
    train=train.to_numpy()
    label=labels(data)
    label=label.iloc[700:]
    return train, label


In [8]:
train, label = preproccess(data)

In [9]:
mean=train.mean(axis=0)
train-=mean
std = train.std(axis=0)
train/=std

In [10]:
test, test_label = preproccess(val)
test-=mean
test/=std

In [12]:
data_tf = {
    'time': train[:,0],
    'inband':train[:,1],
    'outband':train[:,2],
    'latency':train[:,3],
    'packet':train[:,4],
    'jitter': train[:,5]
}

In [16]:
example = tf.train.Example(features=tf.train.Features(feature={
    'time': tf.train.Feature(
        float_list=tf.train.FloatList(value=data_tf['time'])),
    'inband': tf.train.Feature(
        float_list=tf.train.FloatList(value=data_tf['inband'])),
    'outband': tf.train.Feature(
        float_list=tf.train.FloatList(value=data_tf['outband'])),
    'latency': tf.train.Feature(
        float_list=tf.train.FloatList(value=data_tf['latency'])),
    'packet': tf.train.Feature(
        float_list=tf.train.FloatList(value=data_tf['packet'])),
    'jitter': tf.train.Feature(
        float_list=tf.train.FloatList(value=data_tf['jitter'])),
}))

In [ ]:
sess= tf.InteractiveSession()

reader = tf.TFRecordReader()
filename_queue = tf.train.string_input_producer(['processed_train'])

_, serialized_example = reader.read(filename_queue)

read_features = {
    'time':tf.VarLenFeature(dtype=tf.float64),
    'inband':tf.VarLenFeature(dtype=tf.float64),
    'outband':tf.VarLenFeature(dtype=tf.float64),
    'latency':tf.VarLenFeature(dtype=tf.float64),
    'packet':tf.VarLenFeature(dtype=tf.float64),
    'jitter':tf.VarLenFeature(dtype=tf.float64),
}

read_data = tf.parse_single_example(serialized=)

In [ ]:
def get_model():
    model = tf.keras.Sequential([
            tf.keras.layers.Dense(10, activation='relu'),
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Dense(10, activation='relu'),
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Dense(10, activation='relu'),
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Dense(1, activation='sigmoid')])
    model.compile(optimizer='adam',
            loss='BinaryCrossentropy',
            metrics=['accuracy'])    
    return model

In [ ]:
model = get_model()       
history = model.fit(train, label, epochs=3, batch_size=128)

In [ ]:
test_loss, test_acc =model.evaluate(test, test_label, verbose=2)
print('\nTest Accuracy:', test_acc)